In [1]:
import numpy as np

In [2]:
np.__version__

'1.21.2'

##### Points to notify


- slowness of loops
- exploring u functions
    - array arithemetic
    - trignometric functions
    - explonents and logerithms
- advanced u functions
    - specifiying outputs
    - aggregates
    - outer products

##### Computation on NumPy Arrays: Universal Functions

Up until now, we have been discussing some of the basic nuts and bolts of NumPy; in
the next few sections, we will dive into the reasons that NumPy is so important in the
Python data science world. Namely, it provides an easy and flexible interface to opti‐
mized computation with arrays of data. <br>

Computation on NumPy arrays can be very fast, or it can be very slow. The key to
making it fast is to use vectorized operations, generally implemented through Num‐
Py’s universal functions (ufuncs). This section motivates the need for NumPy’s ufuncs,
which can be used to make repeated calculations on array elements much more effi‐
cient. It then introduces many of the most common and useful arithmetic ufuncs
available in the NumPy package.

##### The Slowness of Loops

Python’s default implementation (known as CPython) does some operations very
slowly. This is in part due to the dynamic, interpreted nature of the language: the fact
that types are flexible, so that sequences of operations cannot be compiled down to
efficient machine code as in languages like C and Fortran. Recently there have been
various attempts to address this weakness: well-known examples are the PyPy project,
a just-in-time compiled implementation of Python; the Cython project, which con‐
verts Python code to compilable C code; and the Numba project, which converts
snippets of Python code to fast LLVM bytecode. Each of these has its strengths and
weaknesses, but it is safe to say that none of the three approaches has yet surpassed
the reach and popularity of the standard CPython engine.

The relative sluggishness of Python generally manifests itself in situations where
many small operations are being repeated for instance, looping over arrays to oper
ate on each element. For example, imagine we have an array of values and we’d like to
compute the reciprocal of each. A straightforward approach might look like this:

In [3]:
np.random.seed(0)
def compute_reciprocals(values):
    output = np.empty(len(values))
    for i in range(len(values)):
        output[i] = 1.0 / values[i]
    return output
values = np.random.randint(1, 10, size=5)
compute_reciprocals(values)

array([0.16666667, 1.        , 0.25      , 0.25      , 0.125     ])

This implementation probably feels fairly natural to someone from, say, a C or Java
background. But if we measure the execution time of this code for a large input, we
see that this operation is very slow, perhaps surprisingly so! We’ll benchmark this

In [4]:
big_array = np.random.randint(1, 100, size=1000000)
%timeit compute_reciprocals(big_array)

2.23 s ± 65.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


It takes several seconds to compute these million operations and to store the result!
When even cell phones have processing speeds measured in Giga-FLOPS (i.e., bil‐
lions of numerical operations per second), this seems almost absurdly slow. It turns
out that the bottleneck here is not the operations themselves, but the type-checking
and function dispatches that CPython must do at each cycle of the loop. Each time
the reciprocal is computed, Python first examines the object’s type and does a
dynamic lookup of the correct function to use for that type. If we were working in
compiled code instead, this type specification would be known before the code exe‐
cutes and the result could be computed much more efficiently.

##### Introducing UFuncs

For many types of operations, NumPy provides a convenient interface into just this
kind of statically typed, compiled routine. This is known as a vectorized operation.
You can accomplish this by simply performing an operation on the array, which will
then be applied to each element. This vectorized approach is designed to push the
loop into the compiled layer that underlies NumPy, leading to much faster execution.

In [19]:
np.arange(10)/np.arange(1,11) # should have same size

array([0.        , 0.5       , 0.66666667, 0.75      , 0.8       ,
       0.83333333, 0.85714286, 0.875     , 0.88888889, 0.9       ])

And ufunc operations are not limited to one-dimensional arrays—they can act on
multidimensional arrays as well:

In [20]:
x = np.arange(9).reshape((3, 3))
2 ** x

array([[  1,   2,   4],
       [  8,  16,  32],
       [ 64, 128, 256]], dtype=int32)

Computations using vectorization through ufuncs are nearly always more efficient
than their counterpart implemented through Python loops, especially as the arrays
grow in size. Any time you see such a loop in a Python script, you should consider
whether it can be replaced with a vectorized expression.

##### Exploring NumPy’s UFuncs

Ufuncs exist in two flavors: unary ufuncs, which operate on a single input, and binary
ufuncs, which operate on two inputs. We’ll see examples of both these types of func‐
tions here.

##### Array arithmetic

NumPy’s ufuncs feel very natural to use because they make use of Python’s native
arithmetic operators. The standard addition, subtraction, multiplication, and division
can all be used:

In [21]:
x = np.arange(10)
print("x = ", x)
print("x+4 = ", x+4)
print("x-4 = ", x-4)
print("x*4 = ", x*4)
print("x/4 = ", x/4)

x =  [0 1 2 3 4 5 6 7 8 9]
x+4 =  [ 4  5  6  7  8  9 10 11 12 13]
x-4 =  [-4 -3 -2 -1  0  1  2  3  4  5]
x*4 =  [ 0  4  8 12 16 20 24 28 32 36]
x/4 =  [0.   0.25 0.5  0.75 1.   1.25 1.5  1.75 2.   2.25]


There is also a unary ufunc for negation, a ** operator for exponentiation, and a %
operator for modulus:

In [23]:
print("x ** 4 = ", x**4)
print("x % 4 = ", x%4)

x ** 4 =  [   0    1   16   81  256  625 1296 2401 4096 6561]
x % 4 =  [0 1 2 3 0 1 2 3 0 1]


All of these arithmetic operations are simply convenient wrappers around specific
functions built into NumPy; for example, the + operator is a wrapper for the add
function:

In [24]:
np.add(x,4)

array([ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13])

+ 
np.add
Addition (e.g., 1 + 1 = 2)
-
np.subtract
Subtraction (e.g., 3 - 2 = 1)
-
np.negative
Unary negation (e.g., -2)
*
np.multiply
Multiplication (e.g., 2 * 3 = 6)
/
np.divide
Division (e.g., 3 / 2 = 1.5)
//
np.floor_divide Floor division (e.g., 3 // 2 = 1)
**
np.power
Exponentiation (e.g., 2 ** 3 = 8)
%
np.mod
Modulus/remainder (e.g., 9 % 4 = 1)

##### Trigonometric functions

NumPy provides a large number of useful ufuncs, and some of the most useful for the
data scientist are the trigonometric functions. We’ll start by defining an array of
angles:

In [25]:
theta = np.linspace(0, np.pi, 3)

Now we can compute some trigonometric functions on these values:

In [26]:
print("theta      = ", theta)
print("sin(theta) = ", np.sin(theta))
print("cos(theta) = ", np.cos(theta))
print("tan(theta) = ", np.tan(theta))

theta      =  [0.         1.57079633 3.14159265]
sin(theta) =  [0.0000000e+00 1.0000000e+00 1.2246468e-16]
cos(theta) =  [ 1.000000e+00  6.123234e-17 -1.000000e+00]
tan(theta) =  [ 0.00000000e+00  1.63312394e+16 -1.22464680e-16]


The values are computed to within machine precision, which is why values that
should be zero do not always hit exactly zero. Inverse trigonometric functions are also
available:

In [27]:
x = [-1, 0, 1]
print("x         = ", x)
print("arcsin(x) = ", np.arcsin(x))
print("arccos(x) = ", np.arccos(x))
print("arctan(x) = ", np.arctan(x))

x         =  [-1, 0, 1]
arcsin(x) =  [-1.57079633  0.          1.57079633]
arccos(x) =  [3.14159265 1.57079633 0.        ]
arctan(x) =  [-0.78539816  0.          0.78539816]


##### Exponents and logarithms

Another common type of operation available in a NumPy ufunc are the exponentials:

In [28]:
x = [1, 2, 3]
print("x     =", x)
print("e^x   =", np.exp(x))
print("2^x   =", np.exp2(x))
print("3^x   =", np.power(3, x))

x     = [1, 2, 3]
e^x   = [ 2.71828183  7.3890561  20.08553692]
2^x   = [2. 4. 8.]
3^x   = [ 3  9 27]


The inverse of the exponentials, the logarithms, are also available. The basic np.log
gives the natural logarithm; if you prefer to compute the base-2 logarithm or the
base-10 logarithm, these are available as well:

In [29]:
 x = [1, 2, 4, 10]
print("x        =", x)
print("ln(x)    =", np.log(x))
print("log2(x)  =", np.log2(x))
print("log10(x) =", np.log10(x))

x        = [1, 2, 4, 10]
ln(x)    = [0.         0.69314718 1.38629436 2.30258509]
log2(x)  = [0.         1.         2.         3.32192809]
log10(x) = [0.         0.30103    0.60205999 1.        ]


There are also some specialized versions that are useful for maintaining precision
with very small input:

In [30]:
x = [0, 0.001, 0.01, 0.1]
print("exp(x) - 1 =", np.expm1(x))
print("log(1 + x) =", np.log1p(x))

exp(x) - 1 = [0.         0.0010005  0.01005017 0.10517092]
log(1 + x) = [0.         0.0009995  0.00995033 0.09531018]


When x is very small, these functions give more precise values than if the raw np.log
or np.exp were used.

##### Advanced Ufunc Features

Many NumPy users make use of ufuncs without ever learning their full set of features.
We’ll outline a few specialized features of ufuncs here.

##### Specifying output

For large calculations, it is sometimes useful to be able to specify the array where the
result of the calculation will be stored. Rather than creating a temporary array, you
can use this to write computation results directly to the memory location where you’d 
like them to be. For all ufuncs, you can do this using the out argument of the
function:

In [33]:
x = np.arange(10)
np.multiply(x, 10)

array([ 0, 10, 20, 30, 40, 50, 60, 70, 80, 90])

In [34]:
np.add(x,10)

array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19])

In [35]:
np.divide(x, 10)

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

This can even be used with array views. For example, we can write the results of a
computation to every other element of a specified array:

In [40]:
y = np.zeros(10)
np.power(2, np.arange(5), out=y[::2])
print(y)

[ 1.  0.  2.  0.  4.  0.  8.  0. 16.  0.]


##### Aggregates

For binary ufuncs, there are some interesting aggregates that can be computed
directly from the object. For example, if we’d like to reduce an array with a particular
operation, we can use the reduce method of any ufunc. A reduce repeatedly applies a
given operation to the elements of an array until only a single result remains. <br>

For example, calling reduce on the add ufunc returns the sum of all elements in the


In [42]:
x = np.arange(1, 6)
np.add.reduce(x)

15

In [43]:
np.multiply.reduce(x)

120

In [44]:
np.add.accumulate(x)

array([ 1,  3,  6, 10, 15], dtype=int32)

In [45]:
np.multiply.accumulate(x)

array([  1,   2,   6,  24, 120], dtype=int32)

##### Outer products

Finally, any ufunc can compute the output of all pairs of two different inputs using
the outer method. This allows you, in one line, to do things like create a multiplica‐
tion table:

In [46]:
x = np.arange(1, 6)
np.multiply.outer(x, x)

array([[ 1,  2,  3,  4,  5],
       [ 2,  4,  6,  8, 10],
       [ 3,  6,  9, 12, 15],
       [ 4,  8, 12, 16, 20],
       [ 5, 10, 15, 20, 25]])